### This script encapsulates the processing done to get the final features decided by group 4

In [1]:
import pandas as pd
train_modified_df = pd.read_csv("../../data/train.csv")
train_modified_df.head()

,id,is_appartment,area,added_time,bedrooms,new_building,postcode,lat,lon,advertiser,foto_amount,is_promoted,subtype,sticker,price_drop_date,energy_value,energy_label,province,price
0,tr0,True,88.0,141725167,2.0,0,8300,51.342160,3.287791,Vastio,12.0,1,Appartement,NaN,NaN,210.0,c,West-Vlaanderen,599000.0
1,tr1,True,55.0,132223171,1.0,0,1800,50.933980,4.436390,Ring Consult,10.0,1,Appartement,NaN,NaN,92.0,a,Vlaams-Brabant,199000.0
2,tr2,False,333.0,119151262,4.0,0,3294,50.996456,5.045376,Hillewaere Heist-op-den-Berg,30.0,1,Villa,NaN,2024-06-08 23:29:39,NaN,f,Vlaams-Brabant,875000.0
3,tr3,False,517.0,137313038,5.0,0,1700,50.843364,4.274004,Living Stone Dilbeek,31.0,1,Eengezinswoning,NaN,NaN,322.0,d,Vlaams-Brabant,649000.0
4,tr4,True,99.0,137303955,3.0,0,8300,51.348180,3.276942,Colpin Heist,13.0,1,Appartement,NaN,NaN,161.0,b,West-Vlaanderen,765000.0


In [2]:
# Convert all labels to uppercase to maintain consistency
train_modified_df['energy_label'] = train_modified_df['energy_label'].str.upper()

# Define valid categories that we want to keep
valid_labels = {'A+', 'A', 'B', 'C', 'D', 'E', 'F'}

# Define a function to consolidate variations into their main category
def consolidate_energy_labels(label):
    if label in ['A++']:  
        return 'A+'
    elif label in ['A-']:  
        return 'A'
    elif label in ['B-', 'B+']:  
        return 'B'
    elif label in ['C-', 'C+']:  
        return 'C'
    elif label in ['D-', 'D+']:  
        return 'D'
    elif label in ['E-', 'E+']:  
        return 'E'
    elif label in valid_labels:  # Keep already valid labels
        return label
    else:
        return None  # Remove all other labels

# Apply transformation
train_modified_df['energy_label'] = train_modified_df['energy_label'].apply(consolidate_energy_labels)

In [3]:
# Verify unique values after transformation
print(train_modified_df['energy_label'].value_counts())

energy_label
B     4579
C     4023
F     3413
A     3142
D     3082
E     1828
A+     419
Name: count, dtype: int64


In [4]:
def impute_energy_label_based_on_area(row):
    if pd.isnull(row['energy_label']):
        if row['area'] <= 90:  # Area threshold based on observed median values
            return 'A+'
        elif row['area'] <= 100:
            return 'A'
        elif row['area'] <= 120:
            return 'B'
        elif row['area'] <= 140:
            return 'C'
        elif row['area'] <= 160:
            return 'D'
        elif row['area'] <= 180:
            return 'E'
        else:
            return 'F'
    return row['energy_label']  # Keep existing values if not NaN

train_modified_df['energy_label'] = train_modified_df.apply(impute_energy_label_based_on_area, axis=1)

# Energy value, dealing with imputed value based on energy label
# Step 1: Compute median energy_value for each energy_label, excluding NaNs
energy_median_map = train_modified_df.groupby('energy_label')['energy_value'].median().dropna()

# Step 2: Fill missing energy_value using the median of the corresponding energy_label
train_modified_df['energy_value'] = train_modified_df['energy_value'].fillna(train_modified_df['energy_label'].map(energy_median_map))

# Step 3: Fill remaining NaNs using area-based grouping
area_median_map = train_modified_df.groupby(pd.qcut(train_modified_df['area'], q=4))['energy_value'].median().dropna()
train_modified_df['energy_value'] = train_modified_df['energy_value'].fillna(train_modified_df['area'].map(area_median_map))

# Step 4: Final fallback - fill remaining NaNs with the global median
train_modified_df['energy_value'].fillna(train_modified_df['energy_value'].median(), inplace=True)

/tmp/ipykernel_55618/1762355289.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  area_median_map = train_modified_df.groupby(pd.qcut(train_modified_df['area'], q=4))['energy_value'].median().dropna()
/tmp/ipykernel_55618/1762355289.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_modified_df['energy_value'].fillna(train_modi

In [5]:
# Area - missing 
# Compute median area for each bedroom category
area_median_by_bedrooms = train_modified_df.groupby('bedrooms')['area'].median()

# Fill missing area values based on median area for the same bedroom count
train_modified_df['area'] = train_modified_df.apply(
    lambda row: area_median_by_bedrooms[row['bedrooms']] if pd.isnull(row['area']) else row['area'], 
    axis=1
)

print(train_modified_df['area'].isnull().sum())  # Should be 0

0


In [6]:
# Number of bedrooms
# Define bedroom categories
bins = [0, 1, 4, 8, float('inf')]  # Ranges for categorization
labels = ['Studio/1-Bed', 'Standard (2-4 Beds)', 'Large (5-8 Beds)', 'Luxury (>8 Beds)']

# Apply binning
train_modified_df['bedroom_category'] = pd.cut(train_modified_df['bedrooms'], bins=bins, labels=labels, include_lowest=True)

train_modified_df = train_modified_df.drop(columns=['bedrooms'])


In [7]:
# 1 = Price dropped, 0 = No price drop
train_modified_df['had_price_drop'] = train_modified_df['price_drop_date'].notnull().astype(int)

In [8]:
# Define mapping from old subtypes to new grouped categories
subtype_mapping = {
    'Appartement': 'Apartment', 'Gelijkvloers app.': 'Apartment', 'Duplex': 'Apartment', 
    'Triplex': 'Apartment', 'Loft': 'Apartment', 'Dakappartement': 'Apartment',

    'Studio': 'Studio', 'Studio met slaaphoek': 'Studio', 'Serviceflat': 'Studio', 
    'Assistentie-appartement': 'Studio',

    'Eengezinswoning': 'Single-Family Home', 'Woning': 'Single-Family Home', 'Bel-étage': 'Single-Family Home',
    'Burgerswoning': 'Single-Family Home', 'Arbeiderswoning': 'Single-Family Home', 'Rijwoning': 'Single-Family Home',
    'Hoekwoning': 'Single-Family Home', 'Koppelwoning': 'Single-Family Home',

    'Villa': 'Villa', 'Villa-landhuis': 'Villa', 'Moderne villa': 'Villa', 'Koppelvilla': 'Villa',

    'Herenwoning': 'Luxury Home', 'Herenhuis': 'Luxury Home', 'Uitzonderlijke woning': 'Luxury Home', 
    'Patio woning': 'Luxury Home',

    'Fermette': 'Farmhouse/Rural', 'Hoeve': 'Farmhouse/Rural', 'Boerderij': 'Farmhouse/Rural', 
    'Pastorijwoning': 'Farmhouse/Rural', 'Cottage': 'Farmhouse/Rural',

    'Bungalow': 'Bungalow/Chalet', 'Chalet': 'Bungalow/Chalet', 'Buitenverblijf': 'Bungalow/Chalet', 
    'Vakantiewoning': 'Bungalow/Chalet',

    'Gemengd gebruik': 'Mixed-Use/Commercial', 'App. vrij beroep': 'Mixed-Use/Commercial',

    'Kasteel': 'Unique Properties', 'Woonboot': 'Unique Properties', 'Kangoeroewoning': 'Unique Properties', 
    'Split-level': 'Unique Properties',

    'Andere': 'Other/Unknown', 'Appartementsgebouw': 'Other/Unknown'
}

# Apply mapping
train_modified_df['subtype_category'] = train_modified_df['subtype'].map(subtype_mapping)

# Determine most common subtype for each area range
subtype_mode_by_area = train_modified_df.groupby(pd.qcut(train_modified_df['area'], q=4))['subtype_category'].agg(lambda x: x.mode()[0])

# Fill missing subtype_category based on area
train_modified_df['subtype_category'] = train_modified_df.apply(
    lambda row: subtype_mode_by_area[row['area']] if pd.isnull(row['subtype_category']) else row['subtype_category'],
    axis=1
)

train_modified_df = train_modified_df.drop(columns=['subtype'])

/tmp/ipykernel_55618/1379382074.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  subtype_mode_by_area = train_modified_df.groupby(pd.qcut(train_modified_df['area'], q=4))['subtype_category'].agg(lambda x: x.mode()[0])


In [9]:
# post code
# Convert postcode to string to avoid issues with leading zeros
train_modified_df['postcode'] = train_modified_df['postcode'].astype(str)

# Extract first digit
train_modified_df['postcode_first_digit'] = train_modified_df['postcode'].str[0]

# Extract second and third digits
train_modified_df['postcode_middle_digits'] = train_modified_df['postcode'].str[1:3]

# Extract fourth digit
train_modified_df['postcode_last_digit'] = train_modified_df['postcode'].str[3]

train_modified_df['postcode_first_digit'] = train_modified_df['postcode_first_digit'].astype(int)
train_modified_df['postcode_middle_digits'] = train_modified_df['postcode_middle_digits'].astype(int)
train_modified_df['postcode_last_digit'] = train_modified_df['postcode_last_digit'].astype(int)

train_modified_df = train_modified_df.drop(columns=['postcode'])

In [10]:
# Compute median latitude and longitude for each postcode
lat_median_by_postcode = train_modified_df.groupby('postcode_first_digit')['lat'].median()
lon_median_by_postcode = train_modified_df.groupby('postcode_first_digit')['lon'].median()

# Fill missing latitude values using postcode median
train_modified_df['lat'] = train_modified_df.apply(
    lambda row: lat_median_by_postcode[row['postcode_first_digit']] 
    if pd.isnull(row['lat']) else row['lat'], 
    axis=1
)

# Fill missing longitude values using postcode median
train_modified_df['lon'] = train_modified_df.apply(
    lambda row: lon_median_by_postcode[row['postcode_first_digit']] 
    if pd.isnull(row['lon']) else row['lon'], 
    axis=1
)

train_modified_df['lat'].fillna(train_modified_df['lat'].median(), inplace=True)
train_modified_df['lon'].fillna(train_modified_df['lon'].median(), inplace=True)

print(train_modified_df[['lat', 'lon']].isnull().sum())  # Should be 0

lat    0
lon    0
dtype: int64


/tmp/ipykernel_55618/1409171783.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_modified_df['lat'].fillna(train_modified_df['lat'].median(), inplace=True)
/tmp/ipykernel_55618/1409171783.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col

In [11]:
# Count occurrences of each advertiser
advertiser_counts = train_modified_df['advertiser'].value_counts()

# Identify the top 5 advertisers
top_5_advertisers = advertiser_counts.index[:5].tolist()
print(f"Top 5 Advertisers: {top_5_advertisers}")

def categorize_advertiser(advertiser):
    if pd.isnull(advertiser):  # Handle missing values
        return 'Unknown'
    elif advertiser in top_5_advertisers:
        return 'Well-Known'
    elif advertiser_counts[advertiser] > 50:  # Arbitrary threshold for "Known" category
        return 'Known'
    elif advertiser_counts[advertiser] > 10:  # Arbitrary threshold for "Rare" category
        return 'Rare'
    else:
        return 'Unknown'  # Very rare advertisers are treated as unknown

# Apply function to create a new category column
train_modified_df['advertiser_category'] = train_modified_df['advertiser'].apply(categorize_advertiser)

train_modified_df = train_modified_df.drop(columns=['advertiser'])

print(train_modified_df['advertiser_category'].value_counts())

Top 5 Advertisers: ['NB-Projects', 'Team Construct', 'Immo Da Vinci Gent', 'Hectaar NV', 'Heylen Vastgoed - Turnhout']
advertiser_category
Rare          13894
Unknown        6021
Known          5824
Well-Known     1332
Name: count, dtype: int64


In [12]:
# Define bins and labels
bins = [0, 2, 30, float('inf')]
labels = ['Very Few (0-2)', 'Normal (3-30)', 'High (>30)']

# Apply binning
train_modified_df['foto_category'] = pd.cut(train_modified_df['foto_amount'], bins=bins, labels=labels, include_lowest=True)

train_modified_df = train_modified_df.drop(columns=['is_promoted'])

train_modified_df['is_sticker'] = train_modified_df['sticker'].notnull().astype(int)
train_modified_df = train_modified_df.drop(columns=['sticker'])

In [13]:

# drop price_drop_date
train_modified_df = train_modified_df.drop(columns=['price_drop_date'])

from sklearn.model_selection import train_test_split

# Split data into 80% train and 20% test
train_df, test_df = train_test_split(train_modified_df, test_size=0.2, random_state=42)

# Print sizes to confirm split
print(f"Train set size: {train_df.shape}, Test set size: {test_df.shape}")

Train set size: (21656, 21), Test set size: (5415, 21)
